## 6-3 クラス分類（SVM）

### ライブラリーのインポート

In [ ]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

- sklearn.svm
  - サポートベクターマシン（SVM）のライブラリ
- sklearn.metrics
  - 評価指標を計算する関数群
  - confusion_matrix：混同行列の作成、accuracy_score：正解率の計算

### 機械学習用データの生成

In [ ]:
np.random.seed(7)
data_x = np.arange(-1, 1, 1/float(100)).reshape(-1, 1)  
data_y = np.random.randn(len(data_x), 1) * 0.4
CLASS_RADIUS = 0.6
labels = (np.square(data_x) + np.square(data_y)) < CLASS_RADIUS ** 2

- **data_x**: -1〜1の100個の値
- **data_y**: data_xと同数の乱数（標準正規分布：平均0、分散1）で作成する。
  - 大きさ調整のため0.4を乗じている。
- **CLASS_RADIUS**: 半径
- labels = (np.square(data_x) + np.square(data_y)) < CLASS_RADIUS ** 2
  - data_xの2乗とdata_yの2乗の和が半径の2乗より小さいものを「原点に近いクラス（True）」、大きいものを「原点から遠いクラス（Failes）」とする。

In [ ]:
# データとラベルをPandasデータフレームへ格納（データ表示用）
df = pd.DataFrame(data_x, columns=['data_x'])
df['data_y'] = data_y
df['label'] = labels

# データの表示
display('True（近い）', df.query('label == True').head())
display('False（遠い）', df.query('label == False').head())

- pd.DataFrame(data_x, columns=['data_x'])
  - data_xをPandasのデータフレームへ変換する。
- df.query('label == True').head()
  - ラベルがTrue（原点に近い）クラスのデータの先頭5行を表示する。
- df.query('label == False').head()
  - ラベルがFalse（原点から遠い）クラスのデータの先頭5行を表示する。

### データ散布図の表示

In [ ]:
ax = plt.gca()
df.query('label == True').plot(kind='scatter', 
                               x='data_x', y='data_y', ax=ax, color='red')
df.query('label == False').plot(kind='scatter', 
                                x='data_x', y='data_y', ax=ax)
circle  = plt.Circle((0,0), CLASS_RADIUS, alpha=0.2)
ax.add_patch(circle)
plt.show()

- ax = plt.gca()
  - 現在の描画領域オブジェクトを取得する。
- df.query('label == True').plot(kind='scatter', ･･･)
  - True（近い）クラスのデータを散布図に描画する。
- df.query('label == False').plot(kind='scatter', ･･･)
  - False（遠い）クラスのデータを散布図に描画する。
- plt.Circle((0,0), CLASS_RADIUS, alpha=0.2)
  - クラス境界の円を描画する（alphaは透明度の指定）。

### 訓練／テストデータの分割

In [ ]:
train_x, test_x, train_y, test_y, train_label, test_label = train_test_split(
    data_x, data_y, labels, test_size=0.3, random_state=0)
train_label, test_label = train_label.flatten(), test_label.flatten()

- train_test_split(data_x, data_y, labels, test_size=0.3, random_state=0)
  - 訓練データ80%、テストデータ30%に分割する。
- train_label, test_label = train_label.flatten(), test_label.flatten()
  - ラベルデータを、1次元の配列へ展開する。

### 訓練／テストデータの散布図

In [ ]:
ax = plt.gca()
plt.scatter(train_x, train_y, marker='v', label='Training Data')
plt.scatter(test_x, test_y, marker='^', label='Test Data')
circle = plt.Circle((0,0), CLASS_RADIUS, alpha=0.2)
ax.add_patch(circle)
plt.legend()
plt.show()

- 訓練データを▼、テストデータを▲で表示する。
- plt.Circle((0,0), CLASS_RADIUS, alpha=0.2)
  - 座標(0,0)を中心に、半径「CLASS_RADIUS」の円を描く
  - alpha：透明度の指定

### 機械学習の実行

In [ ]:
# 訓練データとテストデータの配列を作成
train_data = np.c_[train_x, train_y]
test_data = np.c_[test_x, test_y]

# SVM分類器のインスタンス作成と学習実行
clf = svm.SVC(kernel='poly', gamma='auto')
run = clf.fit(train_data, train_label)

- 訓練データ（train_x, train_y）を結合して、train_dataを作成する。
- テストデータ（test_x, test_y）を結合して、test_dataを作成する。
- clf = svm.SVC(kernel='poly', gamma='auto')
  - カーネル関数をpoly（多項式カーネル）として、分類モデルのインスタンスを作成する。
- clf.fit(train_data, train_label.flatten())
  - 訓練データとラベルを入力して機械学習を実行する。

### 学習結果の評価

In [ ]:
# テストデータによる評価
pred = clf.predict(test_data)

# 混同行列と正解率の表示
print('混同行列:\n', confusion_matrix(test_label, pred))
print('正解率:\n', accuracy_score(test_label, pred)) 

- pred = clf.predict(test_data)
  - テストデータ（test_data）を入力して予測結果（pred）を取得する。
- confusion_matrix(test_label, pred)
  - 混同行列を取得する。
- accuracy_score(test_label, pred))
  - 正解率を計算する。